# Assignment 2

Matthew Zlotnik

Robbie Geoghehan

Sebastian Osorio

Shivank Sood

Vikrant Vaidya

### Task A

In [1]:
#!pip install selenium
# !apt-get -q update # to update ubuntu to correctly run apt install
#!apt install -yq chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
import pandas as pd
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') #Ubuntu - Colab
sys.path.insert(0,'C:/Users/sebas/Downloads/chromedriver') #Windows - change user
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [2]:
## Chrome Headless - faster - no new chrome window - it's magic!:
driver = webdriver.Chrome(executable_path='C:/Users/sebas/Downloads/chromedriver', options=chrome_options)

## Regular Selenium - slower - can see what's happening, good for debugging:
#driver = webdriver.Chrome(executable_path='C:/Users/sebas/Downloads/chromedriver')

## Other options:
#driver = webdriver.Chrome('chromedriver', options=chrome_options) #Colab
#driver= webdriver.Chrome() #Windows + PATH

In [3]:
df = pd.DataFrame(columns = ['product_name','product_review','user_rating']) 

In [4]:
# Reach url with top 250 beers
driver.get('https://www.beeradvocate.com/beer/top-rated/')


In [5]:
#Get top 250 beer links
urls =[]
a = driver.find_elements_by_xpath('//*[@id="ba-content"]/table/tbody//td[2]/a')
for x in a:
    urls.append(x.get_attribute('href'))


In [6]:
## for testing with just a couple urls use this:
# for url in urls[5:6]:

## for FULL SCRAPING with ALL URLS use this:
for url in urls:

    driver.get(url)
#    time.sleep(1) # Only use if in slow connection
    reviews = driver.find_elements_by_xpath('//div[@id="rating_fullview"]/div')
    for review in reviews:

        product_name = review.find_element_by_xpath('//h1').text.split('\n')[0]
        user_rating = review.text.split('/')[0]
        
## Use this for List:       
        product_review= review.text.split('\n\n')[1:-2]
        
## Use this for raw text:       
        
#       product_review= review.text #Get ALL TEXT
#       print (len(review.text.split('\n\n'))) # For debugging

        df.loc[len(df)] = [product_name,product_review,user_rating]

In [7]:
df

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,"[2016 Silver Wax. Aroma has whiskey, maple, to...",4.8
1,Kentucky Brunch Brand Stout,[The beer pours Pitch Black with a frothy tan ...,4.74
2,Kentucky Brunch Brand Stout,[Probably the smoothest beer I have ever had. ...,4.68
3,Kentucky Brunch Brand Stout,"[Dark black, very thick, a little bit of tan h...",5
4,Kentucky Brunch Brand Stout,[Poured black as ink with thin ruby edges at 5...,4.97
...,...,...,...
6209,Sure Bet,"[Love their beers, they play more on rhe malt ...",4.34
6210,Sure Bet,"[A - Pours a bright opaque orange with a 1/4"" ...",4.46
6211,Sure Bet,[Appearance - The beer poured a murky orange-y...,4.54
6212,Sure Bet,[Hazy faded gold with a fluffy white head. Sme...,4.38


In [12]:
## Only use if need to change product review from list to text
df['product_review'] = df['product_review'].apply(lambda x: '.'.join(map(str, x)))

In [9]:
#Use if in colab

#from google.colab import drive
#drive.mount('/content/gdrive')

In [13]:
##Only uncomment if want CSV output - careful, could overwrite.

#df.to_csv("reviews_text.csv")


In [1]:
# Library Imports
import spacy
import en_core_web_lg
import pandas as pd
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
nlp = en_core_web_lg.load() #just import once or will take some time
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

## PART B

In [2]:
raw = pd.read_csv("https://raw.githubusercontent.com/coldonline/TextAnalytics/master/HW2/reviews_text.csv")
raw.drop(['Unnamed: 0'], axis=1,inplace=True)
raw.to_csv("raw_scrape.csv")
raw.head()

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,"2016 Silver Wax. Aroma has whiskey, maple, tof...",4.80
1,Kentucky Brunch Brand Stout,The beer pours Pitch Black with a frothy tan h...,4.74
2,Kentucky Brunch Brand Stout,Probably the smoothest beer I have ever had. S...,4.68
3,Kentucky Brunch Brand Stout,"Dark black, very thick, a little bit of tan he...",5.00
4,Kentucky Brunch Brand Stout,Poured black as ink with thin ruby edges at 58...,4.97


In [3]:
# DATA CLEANSING FUNCTIONS

def remove_space(s):
    return s.replace("\n"," ")

def removepunc(item):
    for p in punctuation:
        item = item.lstrip().replace(p,'')
    return item

def lowerize(x):
    return x.lower()

stop_words = set(stopwords.words('english'))

def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 

def lematize(l):
    s=[]
    for i in l:
        s.append(wordnet_lemmatizer.lemmatize(i))
    return s

In [21]:
df = raw.copy()
print(df.shape)

df = df.dropna()
print(df.shape) #finding if there is any major loss due to this

#Cleaning reviews for calculating word freq
df['product_review'] = df['product_review'].apply(remove_space).apply(removepunc).apply(lowerize) 
wordnet_lemmatizer = WordNetLemmatizer()
df['review_words'] = df['product_review'].str.split().apply(set).apply(list).apply(remove_stopwords)
df.head()

(6214, 3)
(6214, 3)


,product_name,product_review,user_rating,review_words
0,Kentucky Brunch Brand Stout,2016 silver wax aroma has whiskey maple toffee...,4.80,"[2016, whiskey, coffee, fruit, barrel, umami, ..."
1,Kentucky Brunch Brand Stout,the beer pours pitch black with a frothy tan h...,4.74,"[coffee, frothy, vanilla, bottle, creams, slic..."
2,Kentucky Brunch Brand Stout,probably the smoothest beer i have ever had sm...,4.68,"[overall, probably, lacking, barrel, excellent..."
3,Kentucky Brunch Brand Stout,dark black very thick a little bit of tan head...,5.00,"[best, coffee, fruit, letdown, probably, end, ..."
4,Kentucky Brunch Brand Stout,poured black as ink with thin ruby edges at 58...,4.97,"[12, superb, legs, fizzy, roast, feel, warming..."


In [5]:
word_bag = []
for i in range(df.shape[0]):
    word_bag += df['review_words'][i]

#word_bag

word_freq = nltk.FreqDist(word_bag)

In [6]:
len(word_freq) # number of words

23894

In [8]:
rslt = pd.DataFrame(word_freq.most_common(22337),              
                    columns=['Word', 'Frequency'])
rslt.head()
rslt.to_csv("beer_review_wordcount.csv")
#some words like 'balanced', 'crisp' , 'robust' seem to be mixing with other words in the file, but that doesn't bother us 
#since we care only about existence than perfect counts

# PART C

In [22]:
# CHOOSING WORDS 'balanced' , 'crisp' , 'robust' they seem pretty prevelant in the data

#spacy implementation


def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
#         if token.lemma_ == '-PRON-'
#             continue
        result.append(token.lemma_)
    return " ".join(result)

def calculate_similarity(text1, text2):
    base = nlp(process_text(text1))
    compare = nlp(process_text(text2))
    return base.similarity(compare)


In [23]:
# Not the cleanest piece of code, but can clean similarity calculation with above function later
# This chunk might take some time
df['similarity_score'] = 0

for i in range(df.shape[0]):
    df.loc[i,'similarity_score'] = nlp(process_text(df.product_review[i])).similarity(nlp("crisp balanced robust"))
df.head()

,product_name,product_review,user_rating,review_words,similarity_score
0,Kentucky Brunch Brand Stout,2016 silver wax aroma has whiskey maple toffee...,4.80,"[2016, whiskey, coffee, fruit, barrel, umami, ...",0.532586
1,Kentucky Brunch Brand Stout,the beer pours pitch black with a frothy tan h...,4.74,"[coffee, frothy, vanilla, bottle, creams, slic...",0.578409
2,Kentucky Brunch Brand Stout,probably the smoothest beer i have ever had sm...,4.68,"[overall, probably, lacking, barrel, excellent...",0.634664
3,Kentucky Brunch Brand Stout,dark black very thick a little bit of tan head...,5.00,"[best, coffee, fruit, letdown, probably, end, ...",0.592787
4,Kentucky Brunch Brand Stout,poured black as ink with thin ruby edges at 58...,4.97,"[12, superb, legs, fizzy, roast, feel, warming...",0.648406


In [24]:
df.drop(['review_words'], axis=1, inplace=True)

In [25]:
df.to_csv('output_partC.csv')

## PART D

In [26]:
#import file with reduced number of reviews - ie 300 reviews that have the highest similarity scores
data = pd.read_csv('output_partC.csv')
data.drop(['Unnamed: 0','user_rating'], axis=1,inplace=True)
data.head()

,product_name,product_review,similarity_score
0,Breakfast Stout,good color no head clean and balanced with an ...,0.765660
1,Flora Plum,2018 releasea pours a very light pale hazy kin...,0.765198
2,Abricot Du Fermier,this is the best sour saison i have ever had n...,0.758598
3,Flora,flora is clean crisp light and tart with smoot...,0.748414
4,Melcher Street (Double Dry-Hopped),canned on 10918pours a bright deeply hazy oran...,0.745546


In [27]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [28]:
##Checking the sentiment
sentiment_analyzer_scores("I hate this product.")
sentiment_analyzer_scores("I love this product!")
sentiment_analyzer_scores("I think this product is okay.")

I hate this product.-------------------- {'neg': 0.649, 'neu': 0.351, 'pos': 0.0, 'compound': -0.5719}
I love this product!-------------------- {'neg': 0.0, 'neu': 0.308, 'pos': 0.692, 'compound': 0.6696}
I think this product is okay.----------- {'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'compound': 0.2263}


In [29]:
data['sentiment_neg'] = data['product_review'].apply(lambda x :analyser.polarity_scores(x)['neg'])
data['sentiment_neu'] = data['product_review'].apply(lambda x :analyser.polarity_scores(x)['neu'])
data['sentiment_pos'] = data['product_review'].apply(lambda x :analyser.polarity_scores(x)['pos'])
data['sentiment_compound'] = data['product_review'].apply(lambda x :analyser.polarity_scores(x)['compound'])

In [30]:
data.head()

,product_name,product_review,similarity_score,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound
0,Breakfast Stout,good color no head clean and balanced with an ...,0.765660,0.053,0.413,0.534,0.9628
1,Flora Plum,2018 releasea pours a very light pale hazy kin...,0.765198,0.023,0.815,0.162,0.9230
2,Abricot Du Fermier,this is the best sour saison i have ever had n...,0.758598,0.000,0.728,0.272,0.9565
3,Flora,flora is clean crisp light and tart with smoot...,0.748414,0.000,0.909,0.091,0.4019
4,Melcher Street (Double Dry-Hopped),canned on 10918pours a bright deeply hazy oran...,0.745546,0.042,0.807,0.151,0.9525


## PART E

In [31]:
recomdat = data.groupby(by = 'product_name').mean()

recbestpos = recomdat.sort_values(by = ['sentiment_compound'], axis = 0, ascending = False)[:1]
reclowestneg = recomdat.sort_values(by = ['sentiment_pos'], axis = 0, ascending = False)[:1]
preferencebeer = recomdat.sort_values(by = ['similarity_score'], axis = 0, ascending = False)[:1]

In [32]:
Recommended = recbestpos.append(reclowestneg)
Recommended = Recommended.append(preferencebeer)
Recommended

,similarity_score,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound
product_name,,,,,
Black & Wild,0.698476,0.006,0.724,0.271,0.9975
It Was All A Dream,0.701630,0.128,0.458,0.414,0.9390
Melcher Street (Double Dry-Hopped),0.745546,0.042,0.807,0.151,0.9525


Our recommended beers are the beers with the highest average compund sentiment, the highest average positive sentiment and the highest combined similarity score of the three important attributes. This provides recommendations to try the two best rated beers by compound sentiment and positive sentiment for someone seeking the best tasting beers in this category as well as a reccomendation for the person seeking a beer with the highest score against the predefined attributes that focuses on meeting their preferences for crisp, balanced and robust beer.

## PART F

#### Check top rated beers

In [33]:
#Calculate average for each beer and show top 3 average scores
df.groupby(["product_name"])["user_rating"].mean().sort_values(ascending=False)[:3]

product_name
Kentucky Brunch Brand Stout    4.812000
Chemtrailmix                   4.811176
Barrel-Aged Abraxas            4.796400
Name: user_rating, dtype: float64

In [42]:
#df["similarity_score"] = (df["similarity_crisp"] + df["similarity_bal"] + df["similarity_rob"]) / 3

In [34]:
#create mask for top 3 beers by average score and calc their similarity scores
mask = (df["product_name"] == "Kentucky Brunch Brand Stout") | (raw["product_name"] == "Chemtrailmix") | (raw["product_name"] == "Barrel-Aged Abraxas")
df[mask].groupby(["product_name"])["similarity_score"].mean().sort_values(ascending=False)

product_name
Chemtrailmix                   0.626864
Barrel-Aged Abraxas            0.617175
Kentucky Brunch Brand Stout    0.594652
Name: similarity_score, dtype: float64

#### Check if they appear in top 300 reviews

In [35]:
mask = data["product_name"] == "Chemtrailmix"
data[mask].groupby(["product_name"])["similarity_score"].mean()

Series([], Name: similarity_score, dtype: float64)

In [36]:
mask = data["product_name"] == "Barrel-Aged Abraxas"
data[mask].groupby(["product_name"])["similarity_score"].mean()

product_name
Barrel-Aged Abraxas    0.723731
Name: similarity_score, dtype: float64

In [37]:
mask = data["product_name"] == "Kentucky Brunch Brand Stout"
data[mask].groupby(["product_name"])["similarity_score"].mean()

Series([], Name: similarity_score, dtype: float64)

#### Compare missing beer with lowest attribute scores of beers in top 300 reviews across the entire scraped set of reviews

In [38]:
#compare with lowest similarity score from above analysis
preferencebeer = recomdat.sort_values(by = ['similarity_score'], axis = 0, ascending = True)[:1]
preferencebeer

,similarity_score,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_compound
product_name,,,,,
Expedition Stout - Bourbon Barrel-Aged,0.695701,0.028,0.838,0.133,0.9678


In [39]:
mask = df["product_name"] == "Expedition Stout - Bourbon Barrel-Aged"
df[mask].groupby(["product_name"])["similarity_score"].mean()

product_name
Expedition Stout - Bourbon Barrel-Aged    0.6213
Name: similarity_score, dtype: float64

#### Conclusion

One of the three top rated beers, Barrel-Aged Abraxas appear in the top 300 reviews based on similarity scores. Therefore this beer would meet the user requirements.

Chemtrailmix may meet the user requirements as it has a higher average similarity score to the lowest average similarity score of beers appearing in the top 300 reviews (based on the average similarity across all reviews) but was not captured in the top 300. It could still be recommended.

Kentucky Brunch Brand Stout does not meet the user requirements based on it not appearing in the top 300 reviews and having an average similarity score below the lowest average similarity score from the 300 reviews.